# Synthetic Data Generation (SDG)


In [1]:
from langchain_community.document_loaders import DirectoryLoader

path = "./data/samples/"
loader = DirectoryLoader(path)
docs = loader.load()

len(docs)

/home/mounty-ed/stuff/agentic_stuff/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parse

2

## Generate Testset


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from ragas.testset import TestsetGenerator
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv(find_dotenv())
api_key = os.getenv("OPENROUTER_API_KEY")

generator_llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    temperature=0.1,
)

generator_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

wrapped_llm = LangchainLLMWrapper(generator_llm)
wrapped_embeddings = LangchainEmbeddingsWrapper(generator_embeddings)

generator = TestsetGenerator.from_langchain(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(
    docs, testset_size=5
)

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/36 [00:00<?, ?it/s]/home/mounty-ed/stuff/agentic_stuff/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Generating Samples: 100%|██████████| 6/6 [00:04<00:00,  1.28it/s]


## Analyze the Testset


In [4]:
df = dataset.to_pandas()
type(df)

pandas.core.frame.DataFrame

In [6]:
df

,user_input,reference_contexts,reference,synthesizer_name
0,What contributions has Han Zhou made to the fi...,[5 2 0 2 b e F 4 ] G L . s c [ 1 v 3 3 5 2 0 ....,Han Zhou is one of the authors of a study that...,single_hop_specifc_query_synthesizer
1,How do Kojima et al. contribute to the underst...,[1. Introduction Exemplar Topology Optimizer d...,Kojima et al. (2022) contribute to the underst...,single_hop_specifc_query_synthesizer
2,What is the significance of prompt design in m...,[2.1. Block-level: Prompt Design for Agents At...,The significance of prompt design in multi-age...,single_hop_specifc_query_synthesizer
3,What were the key factors contributing to Meta...,"[<1-hop>\n\nConclusion In summary, 2024 was a ...",Meta's stock gain of 72% in 2024 was driven by...,multi_hop_specific_query_synthesizer
4,How do the findings of Madaan et al. regarding...,[<1-hop>\n\n2.2. Workflow-level Search Space D...,Madaan et al. emphasize the significance of se...,multi_hop_specific_query_synthesizer
5,What was Amazon's stock performance in 2024 an...,[<1-hop>\n\nStock Market Performance in 2024 U...,Amazon's stock staged an impressive rebound in...,multi_hop_specific_query_synthesizer


## Saving DataFrame as CSV


In [8]:
import pandas as pd

df.to_csv("./data/eval_dataframe.csv", index=False)

loaded_df = pd.read_csv("./data/eval_dataframe.csv")

df.shape, loaded_df.shape

((6, 4), (6, 4))

## Saving DataFrame as Parquet

This format is for larger datasets


In [ ]:
# df.to_parquet("../data/eval_dataframe.parquet")

# loaded_df = pd.read_parquet("../data/eval_dataframe.parquet")

# df.shape, loaded_df.shape
# ## Saving DataFrame as Feather


((12, 4), (12, 4))